# FETCHING DATA FROM YOUTUBE IN YOUTUBE SONG

In [ ]:
!pip install s3fs

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('s3://full-stack-bigdata-datasets/Big_Data/youtube_playlog.csv')

In [4]:
df.head()

,timestamp,user,song
0,1392387533,0,t1l8Z6gLPzo
1,1392387538,1,t1l8Z6gLPzo
2,1392387556,2,t1l8Z6gLPzo
3,1392387561,3,we5gzZq5Avg
4,1392387566,4,we5gzZq5Avg


In [6]:
df.duplicated().sum()

123651

Cleaning dataframe by removing duplicates from song and create a list of songs.

In [9]:
song_id = df.song.drop_duplicates()

numbers of songs or video in list :

In [10]:
song_id.count()

631348

**Estimation of required api calls and time in hours for collecting data**

In [12]:
required_api_calls = (631348 // 50)
time_in_hours = (required_api_calls * 2)/3600

print(f'We need to do {required_api_calls} API calls to fetch all the data.')
print(f'It would take about {time_in_hours:.1f} hours to complete.')

We need to do 12626 API calls to fetch all the data.
It would take about 7.0 hours to complete.


To collect we will use an asynchrone scrypt to be faster

**Estimation extraction's quantity data with the limit set by google (10 000 / days)**

In [15]:
max_calls = 10000
max_videos_count = max_calls * 50

print(f'We can perform {max_calls} calls a day before reaching the API limit.')
print(f'That means we can fetch extra data for {max_videos_count} videos per day.')

We can perform 10000 calls a day before reaching the API limit.
That means we can fetch extra data for 500000 videos per day.


**Fetching Data**

*we are going to fetch only 500 data for training*

In [20]:
songs_sample = song_id[0:500]
songs_sample.count()

500

In [17]:
YOUTUBE_API_KEY = 'xxxxxxxx' 

import os
os.environ['YOUTUBE_API_KEY'] = YOUTUBE_API_KEY

In [18]:
from data_collection import fetch_all 

In [21]:
import asyncio
loop = asyncio.get_event_loop()

task = loop.create_task(fetch_all(songs_sample, dry_run=False))

[2020-11-16 08:53:41 UTC]	INFO	Requesting data for 500 videos. Please wait...	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	DEBUG	API call succeeded.	(data_collection)
[2020-11-16 08:53:42 UTC]	INFO	Done! Fetched data from 500 videos in 1.65 sec	(data_collection)


In [23]:
import json

result = task.result()
print(len(result))

10


*Dumping result in json files*

In [25]:
json_result = json.dumps(result)

**Storing Data in S3**

In [ ]:
import os
import logging

import boto3

logging.getLogger('botocore').setLevel(logging.INFO)
logging.getLogger('boto3').setLevel(logging.INFO)

In [ ]:
ACCESS_KEY_ID = 'xxxxxx'
SECRET_ACCESS_KEY = 'xxxxxx'

session = boto3.Session(ACCESS_KEY_ID, SECRET_ACCESS_KEY)

In [ ]:
s3 = session.resource('s3')

bucket = s3.create_bucket(Bucket = "xxxxxxxxxxxx")

put_object = bucket.put_object ( Key = "songs.json", Body = json_result) 